In [16]:
#This model help bank to predict if a user is going to leave them
#an ANN is used
#Some machine learning techniques have been performed

In [1]:

# Import libraries
import pandas as pd
import numpy as np

# Import data
dataset = pd.read_csv('Churn_Modelling.csv')

X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

# Encode categorical data and scale continuous data
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
preprocess = make_column_transformer(
        (OneHotEncoder(), ['Geography', 'Gender']),
        (StandardScaler(), ['CreditScore', 'Age', 'Tenure', 'Balance',
                            'NumOfProducts', 'HasCrCard', 'IsActiveMember', 
                            'EstimatedSalary']))
X = preprocess.fit_transform(X)
X = np.delete(X, [0,3], 1)

# Split in train/test
y = y.values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 0s 460us/step - loss: 0.4809 - accuracy: 0.7958
Epoch 2/100
800/800 [==============================] - 0s 453us/step - loss: 0.4295 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 0s 451us/step - loss: 0.4226 - accuracy: 0.8021
Epoch 4/100
800/800 [==============================] - 0s 450us/step - loss: 0.4179 - accuracy: 0.8266
Epoch 5/100
800/800 [==============================] - 0s 449us/step - loss: 0.4152 - accuracy: 0.8292
Epoch 6/100
800/800 [==============================] - 0s 451us/step - loss: 0.4139 - accuracy: 0.8331
Epoch 7/100
800/800 [==============================] - 0s 449us/step - loss: 0.4124 - accuracy: 0.8326
Epoch 8/100
800/800 [==============================] - 0s 447us/step - loss: 0.4110 - accuracy: 0.8339
Epoch 9/100
800/800 [==============================] - 0s 451us/step - loss: 0.4104 - accuracy: 0.8339
Epoch 10/100
800/800 [==============================] - 0s 452us/step - l

800/800 [==============================] - 0s 448us/step - loss: 0.4007 - accuracy: 0.8359
Epoch 80/100
800/800 [==============================] - 0s 461us/step - loss: 0.4007 - accuracy: 0.8344
Epoch 81/100
800/800 [==============================] - 0s 449us/step - loss: 0.4007 - accuracy: 0.8346
Epoch 82/100
800/800 [==============================] - 0s 488us/step - loss: 0.4008 - accuracy: 0.8349
Epoch 83/100
800/800 [==============================] - 0s 475us/step - loss: 0.4010 - accuracy: 0.8353
Epoch 84/100
800/800 [==============================] - 0s 460us/step - loss: 0.4007 - accuracy: 0.8361
Epoch 85/100
800/800 [==============================] - 0s 465us/step - loss: 0.4004 - accuracy: 0.8354
Epoch 86/100
800/800 [==============================] - 0s 438us/step - loss: 0.4006 - accuracy: 0.8347
Epoch 87/100
800/800 [==============================] - 0s 438us/step - loss: 0.4006 - accuracy: 0.8351
Epoch 88/100
800/800 [==============================] - 0s 443us/step - loss:

In [2]:

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)


In [3]:
Xnew = pd.DataFrame(data={
        'CreditScore': [600], 
        'Geography': ['France'], 
        'Gender': ['Male'],
        'Age': [40],
        'Tenure': [3],
        'Balance': [60000],
        'NumOfProducts': [2],
        'HasCrCard': [1],
        'IsActiveMember': [1],
        'EstimatedSalary': [50000]})
Xnew = preprocess.transform(Xnew)
Xnew = np.delete(Xnew, [0,3], 1)
new_prediction = classifier.predict(Xnew)
new_prediction = (new_prediction > 0.5)

In [4]:
new_prediction

array([[False]])

In [6]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [7]:
cm

array([[1544,   51],
       [ 269,  136]])

In [8]:
# Evaluate
def build_classifier(optimizer='adam'):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

# Evaluate
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, 
                             cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [12]:
mean

0.8464999973773957

In [11]:
variance

0.017720043804567362

In [13]:
# Tune
from sklearn.model_selection import GridSearchCV
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/100
288/288 [==============================] - 0s 517us/step - loss: 0.5730 - accuracy: 0.7954
Epoch 2/100
288/288 [==============================] - 0s 494us/step - loss: 0.4391 - accuracy: 0.8012
Epoch 3/100
288/288 [==============================] - 0s 476us/step - loss: 0.4280 - accuracy: 0.8163
Epoch 4/100
288/288 [==============================] - 0s 464us/step - loss: 0.4216 - accuracy: 0.8225
Epoch 5/100
288/288 [==============================] - 0s 475us/step - loss: 0.4179 - accuracy: 0.8260
Epoch 6/100
288/288 [==============================] - 0s 452us/step - loss: 0.4145 - accuracy: 0.8275
Epoch 7/100
288/288 [==============================] - 0s 452us/step - loss: 0.4122 - accuracy: 0.8289
Epoch 8/100
288/288 [==============================] - 0s 457us/step - loss: 0.4099 - accuracy: 0.8299
Epoch 9/100
288/288 [==============================] - 0s 452us/step - loss: 0.4081 - accuracy: 0.8297
Epoch 10/100
288/288 [==============================] - 0s 456us/step - l

288/288 [==============================] - 0s 509us/step - loss: 0.3421 - accuracy: 0.8615
Epoch 80/100
288/288 [==============================] - 0s 495us/step - loss: 0.3412 - accuracy: 0.8593
Epoch 81/100
288/288 [==============================] - 0s 492us/step - loss: 0.3418 - accuracy: 0.8579
Epoch 82/100
288/288 [==============================] - 0s 499us/step - loss: 0.3408 - accuracy: 0.8603
Epoch 83/100
288/288 [==============================] - 0s 464us/step - loss: 0.3408 - accuracy: 0.8601
Epoch 84/100
288/288 [==============================] - 0s 476us/step - loss: 0.3414 - accuracy: 0.8582
Epoch 85/100
288/288 [==============================] - 0s 473us/step - loss: 0.3403 - accuracy: 0.8619
Epoch 86/100
288/288 [==============================] - 0s 475us/step - loss: 0.3401 - accuracy: 0.8582
Epoch 87/100
288/288 [==============================] - 0s 474us/step - loss: 0.3413 - accuracy: 0.8594
Epoch 88/100
288/288 [==============================] - 0s 478us/step - loss:

288/288 [==============================] - 0s 459us/step - loss: 0.4011 - accuracy: 0.8367
Epoch 53/100
288/288 [==============================] - 0s 463us/step - loss: 0.4006 - accuracy: 0.8343
Epoch 54/100
288/288 [==============================] - 0s 460us/step - loss: 0.4009 - accuracy: 0.8358
Epoch 55/100
288/288 [==============================] - 0s 460us/step - loss: 0.4006 - accuracy: 0.8364
Epoch 56/100
288/288 [==============================] - 0s 459us/step - loss: 0.4009 - accuracy: 0.8363
Epoch 57/100
288/288 [==============================] - 0s 460us/step - loss: 0.4007 - accuracy: 0.8368
Epoch 58/100
288/288 [==============================] - 0s 456us/step - loss: 0.4005 - accuracy: 0.8358
Epoch 59/100
288/288 [==============================] - 0s 461us/step - loss: 0.4005 - accuracy: 0.8356
Epoch 60/100
288/288 [==============================] - 0s 458us/step - loss: 0.4000 - accuracy: 0.8367
Epoch 61/100
288/288 [==============================] - 0s 464us/step - loss:

Epoch 31/100
288/288 [==============================] - 0s 457us/step - loss: 0.4056 - accuracy: 0.8351
Epoch 32/100
288/288 [==============================] - 0s 461us/step - loss: 0.4050 - accuracy: 0.8347
Epoch 33/100
288/288 [==============================] - 0s 452us/step - loss: 0.4049 - accuracy: 0.8347
Epoch 34/100
288/288 [==============================] - 0s 459us/step - loss: 0.4050 - accuracy: 0.8336
Epoch 35/100
288/288 [==============================] - 0s 457us/step - loss: 0.4043 - accuracy: 0.8363
Epoch 36/100
288/288 [==============================] - 0s 457us/step - loss: 0.4047 - accuracy: 0.8354
Epoch 37/100
288/288 [==============================] - 0s 458us/step - loss: 0.4044 - accuracy: 0.8349
Epoch 38/100
288/288 [==============================] - 0s 460us/step - loss: 0.4040 - accuracy: 0.8360
Epoch 39/100
288/288 [==============================] - 0s 459us/step - loss: 0.4038 - accuracy: 0.8354
Epoch 40/100
288/288 [==============================] - 0s 462us

288/288 [==============================] - 0s 457us/step - loss: 0.4141 - accuracy: 0.8283
Epoch 10/100
288/288 [==============================] - 0s 455us/step - loss: 0.4130 - accuracy: 0.8279
Epoch 11/100
288/288 [==============================] - 0s 458us/step - loss: 0.4117 - accuracy: 0.8314
Epoch 12/100
288/288 [==============================] - 0s 452us/step - loss: 0.4107 - accuracy: 0.8335
Epoch 13/100
288/288 [==============================] - 0s 454us/step - loss: 0.4098 - accuracy: 0.8324
Epoch 14/100
288/288 [==============================] - 0s 456us/step - loss: 0.4092 - accuracy: 0.8338
Epoch 15/100
288/288 [==============================] - 0s 451us/step - loss: 0.4086 - accuracy: 0.8351
Epoch 16/100
288/288 [==============================] - 0s 455us/step - loss: 0.4081 - accuracy: 0.8343
Epoch 17/100
288/288 [==============================] - 0s 452us/step - loss: 0.4072 - accuracy: 0.8353
Epoch 18/100
288/288 [==============================] - 0s 461us/step - loss:

...

320/320 [==============================] - 0s 480us/step - loss: 0.3940 - accuracy: 0.8370
Epoch 125/500
320/320 [==============================] - 0s 486us/step - loss: 0.3942 - accuracy: 0.8369
Epoch 126/500
320/320 [==============================] - 0s 480us/step - loss: 0.3942 - accuracy: 0.8380
Epoch 127/500
320/320 [==============================] - 0s 487us/step - loss: 0.3942 - accuracy: 0.8375
Epoch 128/500
320/320 [==============================] - 0s 476us/step - loss: 0.3940 - accuracy: 0.8367
Epoch 129/500
320/320 [==============================] - 0s 478us/step - loss: 0.3940 - accuracy: 0.8371
Epoch 130/500
320/320 [==============================] - 0s 485us/step - loss: 0.3938 - accuracy: 0.8378
Epoch 131/500
320/320 [==============================] - 0s 475us/step - loss: 0.3940 - accuracy: 0.8371
Epoch 132/500
320/320 [==============================] - 0s 483us/step - loss: 0.3937 - accuracy: 0.8379
Epoch 133/500
320/320 [==============================] - 0s 477us/ste

320/320 [==============================] - 0s 465us/step - loss: 0.3771 - accuracy: 0.8409
Epoch 203/500
320/320 [==============================] - 0s 462us/step - loss: 0.3729 - accuracy: 0.8447
Epoch 204/500
320/320 [==============================] - 0s 465us/step - loss: 0.3689 - accuracy: 0.8446
Epoch 205/500
320/320 [==============================] - 0s 463us/step - loss: 0.3651 - accuracy: 0.8453
Epoch 206/500
320/320 [==============================] - 0s 462us/step - loss: 0.3620 - accuracy: 0.8486
Epoch 207/500
320/320 [==============================] - 0s 465us/step - loss: 0.3593 - accuracy: 0.8503
Epoch 208/500
320/320 [==============================] - 0s 461us/step - loss: 0.3571 - accuracy: 0.8503
Epoch 209/500
320/320 [==============================] - 0s 464us/step - loss: 0.3546 - accuracy: 0.8545
Epoch 210/500
320/320 [==============================] - 0s 465us/step - loss: 0.3532 - accuracy: 0.8554
Epoch 211/500
320/320 [==============================] - 0s 463us/ste

320/320 [==============================] - 0s 466us/step - loss: 0.3352 - accuracy: 0.8608
Epoch 281/500
320/320 [==============================] - 0s 464us/step - loss: 0.3360 - accuracy: 0.8631
Epoch 282/500
320/320 [==============================] - 0s 466us/step - loss: 0.3356 - accuracy: 0.8637
Epoch 283/500
320/320 [==============================] - 0s 462us/step - loss: 0.3359 - accuracy: 0.8606
Epoch 284/500
320/320 [==============================] - 0s 466us/step - loss: 0.3354 - accuracy: 0.8610
Epoch 285/500
320/320 [==============================] - 0s 459us/step - loss: 0.3355 - accuracy: 0.8629
Epoch 286/500
320/320 [==============================] - 0s 467us/step - loss: 0.3356 - accuracy: 0.8636
Epoch 287/500
320/320 [==============================] - 0s 466us/step - loss: 0.3361 - accuracy: 0.8629
Epoch 288/500
320/320 [==============================] - 0s 463us/step - loss: 0.3356 - accuracy: 0.8621
Epoch 289/500
320/320 [==============================] - 0s 495us/ste

320/320 [==============================] - 0s 459us/step - loss: 0.3336 - accuracy: 0.8622
Epoch 359/500
320/320 [==============================] - 0s 465us/step - loss: 0.3340 - accuracy: 0.8629
Epoch 360/500
320/320 [==============================] - 0s 465us/step - loss: 0.3340 - accuracy: 0.8636
Epoch 361/500
320/320 [==============================] - 0s 464us/step - loss: 0.3336 - accuracy: 0.8618
Epoch 362/500
320/320 [==============================] - 0s 465us/step - loss: 0.3338 - accuracy: 0.8649
Epoch 363/500
320/320 [==============================] - 0s 467us/step - loss: 0.3339 - accuracy: 0.8627
Epoch 364/500
320/320 [==============================] - 0s 465us/step - loss: 0.3339 - accuracy: 0.8634
Epoch 365/500
320/320 [==============================] - 0s 459us/step - loss: 0.3339 - accuracy: 0.8646
Epoch 366/500
320/320 [==============================] - 0s 463us/step - loss: 0.3339 - accuracy: 0.8645
Epoch 367/500
320/320 [==============================] - 0s 463us/ste

320/320 [==============================] - 0s 484us/step - loss: 0.3336 - accuracy: 0.8622
Epoch 437/500
320/320 [==============================] - 0s 476us/step - loss: 0.3338 - accuracy: 0.8625
Epoch 438/500
320/320 [==============================] - 0s 482us/step - loss: 0.3335 - accuracy: 0.8626
Epoch 439/500
320/320 [==============================] - 0s 482us/step - loss: 0.3337 - accuracy: 0.8629
Epoch 440/500
320/320 [==============================] - 0s 475us/step - loss: 0.3337 - accuracy: 0.8622
Epoch 441/500
320/320 [==============================] - 0s 486us/step - loss: 0.3337 - accuracy: 0.8627
Epoch 442/500
320/320 [==============================] - 0s 479us/step - loss: 0.3335 - accuracy: 0.8626
Epoch 443/500
320/320 [==============================] - 0s 486us/step - loss: 0.3327 - accuracy: 0.8627
Epoch 444/500
320/320 [==============================] - 0s 477us/step - loss: 0.3337 - accuracy: 0.8615
Epoch 445/500
320/320 [==============================] - 0s 473us/ste

In [14]:
best_parameters

{'batch_size': 25, 'epochs': 500, 'optimizer': 'rmsprop'}

In [15]:
best_accuracy

0.8525